In [ ]:
%cd /home/druhe/rail1/src/cnf
import os
os.environ['DATAROOT'] = '/home/druhe/datasets'



/home/druhe/rail1/src/cnf


/home/druhe/rail1/src/cnf/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [60]:
import wandb
import rail1.run
import rail1.checkpoint
import tempfile
import models
import torch
import torch.nn.functional as F

torch.set_grad_enabled(False)

# TODO: set seeds etc, just like training.

In [61]:
api = wandb.Api()
path = 'druhe/cnf/9utazgqo'
run = api.run(path)

run_dir = os.path.join('runs', 'notebooks', run.id)

os.makedirs(run_dir, exist_ok=True)

In [62]:

artifact_dict = {}

for a in run.logged_artifacts():

    if a.type != "checkpoint":
        continue

    file = a.metadata['filename']
    

    metrics = file.rsplit('/')[-1]
    metrics = metrics.split('-')

    dictionary = {item.split('=')[0]: item.split('=')[1] for item in metrics}
    dictionary['artifact'] = a

    artifact_dict[a] = dictionary


In [63]:
best_artifact = min(artifact_dict, key=lambda x: artifact_dict[x]['val_loss'])

In [64]:
best_artifact.download(root=os.path.join(run_dir, 'checkpoints'))

wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


'runs/notebooks/9utazgqo/checkpoints'

In [65]:
config = run.config

In [66]:
model = getattr(models, config["model"].pop("name"))(**config["model"])




In [67]:
train_state = {}
optimizer = None
rail1.checkpoint.load_checkpoint(os.path.join(run_dir, 'checkpoints'), model, train_state=train_state, optimizer=optimizer)


Successfully restored complete state from: runs/notebooks/9utazgqo/checkpoints/step=73728-epoch=0-val_loss=0.0026-val_s_it=0.0063.pt



{'global_step': 73728,
 'last_global_step': 73728,
 'should_raise': None,
 'current_epoch': 0,
 'device': device(type='cuda', index=0),
 'train_metrics': defaultdict(list, {}),
 'total_parameters': 1132944,
 'batch_index': 157}

In [68]:
dataset_config = config["dataset"]
data = getattr(rail1.datasets, dataset_config.pop("name"))(**dataset_config)


In [69]:
images = torch.stack([data['train_dataset'][i][0] for i in range(len(data['train_dataset']))])

In [70]:
images = images.view(len(images), -1)

In [71]:
z = model.encoder(images)

In [73]:
save_dir = os.path.join(run_dir, 'latent_space')
os.makedirs(save_dir, exist_ok=True)
torch.save(z, os.path.join(save_dir, 'train.pt'))



In [74]:
images = torch.stack([data['test_dataset'][i][0] for i in range(len(data['test_dataset']))])
images = images.view(len(images), -1)
z = model.encoder(images)
torch.save(z, os.path.join(save_dir, 'val.pt'))
